# Initialization

In [1]:
%load_ext autoreload
%load_ext line_profiler
import sys
sys.path.append("..")

In [27]:
%autoreload

import numpy as np
import pickle

import main


import matplotlib
import matplotlib.pyplot as plt
matplotlib.rc_file('matplotlibrc')

%matplotlib inline

matplotlib.rcParams['figure.figsize'] = [10,10]


import darkhistory.physics as phys
import darkhistory.utilities as utils
import darkhistory.spec.transferfunction as tf
import darkhistory.spec.spectools as spectools

from darkhistory.spec.spectrum import Spectrum

from darkhistory.electrons.ics.ics_spectrum import ics_spec
from darkhistory.electrons.ics.ics_engloss_spectrum import engloss_spec
from darkhistory.electrons.elec_cooling import get_elec_cooling_tf_fast

from darkhistory.electrons import positronium

from scipy.interpolate import interp1d
from scipy.linalg import solve_triangular

from astropy.io import fits


In [3]:
ics_thomson_ref_tf=pickle.load(open("/Users/hongwan/Dropbox (MIT)/Photon Deposition/tflists/ics/ics_thomson_ref_tf.raw", "rb")) 
ics_rel_ref_tf=pickle.load(open("/Users/hongwan/Dropbox (MIT)/Photon Deposition/tflists/ics/ics_rel_ref_tf.raw", "rb"))
engloss_ref_tf=pickle.load(open("/Users/hongwan/Dropbox (MIT)/Photon Deposition/tflists/ics/engloss_ref_tf.raw", "rb"))

In [34]:
%autoreload

# eleceng = 10**np.arange(0, np.log10(5e12), 0.0254)
# photeng = 10**np.arange(-4, np.log10(5e12), 0.0334)

hdul = fits.open(
    '/Users/hongwan/Dropbox (MIT)/Photon Deposition/idl_comparison/final_darkhistory_comparison/'
    +'tf_elec_z_3.000E+03_nstep_6622_logE_6.477E+00_xe_0.000E+00.fits'
)

data = hdul[1].data

eleceng = data.field('eleceng')[0]
photeng = data.field('energy')[0]


rs = 3000
xH = 1

from darkhistory.spec.spectra import Spectra

coll_ion_sec_elec_specs = (
    phys.coll_ion_sec_elec_spec(eleceng, eleceng, species='HI'),
    phys.coll_ion_sec_elec_spec(eleceng, eleceng, species='HeI'),
    phys.coll_ion_sec_elec_spec(eleceng, eleceng, species='HeII')
)

coll_exc_sec_elec_tf_HI = tf.TransFuncAtRedshift(
    np.squeeze(np.identity(eleceng.size)[:, np.where(eleceng > phys.lya_eng)]),
    in_eng = eleceng, rs = rs*np.ones_like(eleceng), 
    eng = eleceng[eleceng > phys.lya_eng] - phys.lya_eng,
    dlnz = -1, spec_type = 'N'
)

coll_exc_sec_elec_tf_HeI = tf.TransFuncAtRedshift(
    np.squeeze(np.identity(eleceng.size)[:, np.where(eleceng > phys.He_exc_eng)]),
    in_eng = eleceng, rs = rs*np.ones_like(eleceng), 
    eng = eleceng[eleceng > phys.He_exc_eng] - phys.He_exc_eng,
    dlnz = -1, spec_type = 'N'
)

coll_exc_sec_elec_tf_HeII = tf.TransFuncAtRedshift(
    np.squeeze(np.identity(eleceng.size)[:, np.where(eleceng > 4*phys.lya_eng)]),
    in_eng = eleceng, rs = rs*np.ones_like(eleceng), 
    eng = eleceng[eleceng > 4*phys.lya_eng] - 4*phys.lya_eng,
    dlnz = -1, spec_type = 'N'
)

coll_exc_sec_elec_tf_HI.rebin(eleceng)
coll_exc_sec_elec_tf_HeI.rebin(eleceng)
coll_exc_sec_elec_tf_HeII.rebin(eleceng)

coll_exc_sec_elec_specs = (
    coll_exc_sec_elec_tf_HI.grid_vals,
    coll_exc_sec_elec_tf_HeI.grid_vals,
    coll_exc_sec_elec_tf_HeII.grid_vals
)

from darkhistory.spec.spectools import EnglossRebinData

ics_engloss_data = EnglossRebinData(eleceng, photeng, eleceng)


../darkhistory/spec/spectra.py:401: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  np.issubdtype(type(other), float)
../darkhistory/spec/spectra.py:402: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  or np.issubdtype(type(other), int)


# With Linear Algebra

In [35]:
def get_elec_cooling_tf_fast_linalg(
    raw_nonrel_tf, raw_rel_tf, raw_engloss_tf,
    coll_ion_sec_elec_specs, coll_exc_sec_elec_specs,
    eleceng, photeng, rs, xH, xHe=0, ics_engloss_data=None, 
    check_conservation_eng = False, verbose=False
):

    """Returns transfer function for complete electron cooling through ICS and atomic processes.

    Parameters
    ----------
    nonrel_tf : TransFuncAtRedshift
        Raw nonrelativistic primary electron ICS transfer function.
    rel_tf : string
        Raw relativistic primary electron ICS transfer function.
    engloss_tf_filename : string
        Raw primary electron ICS energy loss transfer function.
    coll_ion_sec_elec_specs : tuple of ndarray
        Normalized collisional ionization secondary electron spectra, order HI, HeI, HeII, indexed by eleceng (injection) x eleceng (abscissa).
    coll_exc_sec_elec_specs : tuple of ndarray
        Normalized collisional excitation secondary electron spectra, order HI, HeI, HeII, indexed by eleceng (injection) x eleceng (abscissa).
    eleceng : ndarray
        The electron *kinetic* energy abscissa.
    photeng : ndarray
        The photon energy abscissa.
    rs : float
        The redshift.
    xe : float
        Free electron fraction. 
    xHe : float, optional
        Singly-ionized helium fraction, nHe+/nH. Set to nHe/nH*xe if None.
    ics_engloss_data : EnglossRebinData
        Stores rebinning information for speed. 
    check_conservation_eng : bool
        If true, checks for energy conservation.
    verbose : bool
        If true, prints energy conservation checks.
    
    Returns
    -------

    tuple of TransFuncAtRedshift
        Transfer functions for photons and low energy electrons.

    Note
    ----
    The raw transfer functions should be generated when the code package is first installed. The transfer function corresponds to the fully resolved
    photon spectrum after scattering by one electron.

    This version of the code works faster, but dispenses with energy conservation checks and several other safeguards. Use only with default abscissa, or when get_ics_cooling_tf works.

    """
    xe = xH + xHe
    
    # v/c of electrons, important subsequently.
    beta_ele = np.sqrt(1 - 1/(1 + eleceng/phys.me)**2)
        
    #####################################
    # Inverse Compton
    #####################################

    T = phys.TCMB(rs)

    # Photon transfer function for single primary electron single scattering.
    # This is dN/(dE dt), dt = 1 s.
    phot_ICS_tf = ics_spec(
        eleceng, photeng, T, nonrel_tf = raw_nonrel_tf, rel_tf = raw_rel_tf
    )

    # Downcasting speeds up np.dot
    phot_ICS_tf._grid_vals = phot_ICS_tf.grid_vals.astype('float64')

    # Energy loss transfer function for single primary electron
    # single scattering. This is dN/(dE dt), dt = 1 s.
    engloss_ICS_tf = engloss_spec(
        eleceng, photeng, T, nonrel_tf = raw_engloss_tf, rel_tf = raw_rel_tf
    )

    # Downcasting speeds up np.dot
    engloss_ICS_tf._grid_vals = engloss_ICS_tf.grid_vals.astype('float64')

    # Switch the spectra type here to type 'N'.
    if phot_ICS_tf.spec_type == 'dNdE':
        phot_ICS_tf.switch_spec_type()
    if engloss_ICS_tf.spec_type == 'dNdE':
        engloss_ICS_tf.switch_spec_type()


    # Define some useful lengths.
    N_eleceng = eleceng.size
    N_photeng = photeng.size

    # Create the secondary electron transfer functions.

    # ICS transfer function.
    elec_ICS_tf = tf.TransFuncAtRedshift(
        np.zeros((N_eleceng, N_eleceng)), in_eng = eleceng,
        rs = rs*np.ones_like(eleceng), eng = eleceng,
        dlnz = -1, spec_type = 'N'
    )

    if ics_engloss_data is not None:
        elec_ICS_tf._grid_vals = ics_engloss_data.rebin(
            engloss_ICS_tf.grid_vals
        )
    else:
        elec_ICS_tf._grid_vals = spectools.engloss_rebin_fast(
            eleceng, photeng, engloss_ICS_tf.grid_vals, eleceng
        )
    
    # Total upscattered photon energy.
    cont_loss_ICS_vec = np.zeros_like(eleceng)
    # Deposited energy, enforces energy conservation.
    deposited_ICS_vec = np.zeros_like(eleceng)
    
    
    #####################
    # Excitation  
    #####################
    
    # Construct the rate matrices first. Secondary electron spectrum is an electron at in_eng - excitation energy, 
    # with a per second rate given by n*sigma*c.
    

    # rate_matrix_exc_HI = np.diag(
    #     (1 - xe)*phys.nH*rs**3 * phys.coll_exc_xsec(eleceng, species='HI') * beta_ele * phys.c
    # )
    
    # rate_matrix_exc_HeI = np.diag(
    #     (phys.nHe/phys.nH - xHe)*phys.nH*rs**3 * phys.coll_exc_xsec(eleceng, species='HeI') * beta_ele * phys.c
    # )
    
    # rate_matrix_exc_HeII = np.diag(
    #     xHe*phys.nH*rs**3 * phys.coll_exc_xsec(eleceng, species='HeII') * beta_ele * phys.c
    # )

    # Construct the TransFuncAtRedshift objects.
    # Electrons scatter from in_eng to in_eng - excitation energy.
    # Remove all of the columns (eng) that have energies below the excitation energy, 
    # elec_exc_HI_tf = tf.TransFuncAtRedshift(
    #     np.squeeze(rate_matrix_exc_HI[:, np.where(eleceng > phys.lya_eng)]), 
    #     in_eng = eleceng, rs = rs*np.ones_like(eleceng), 
    #     eng = eleceng[eleceng > phys.lya_eng] - phys.lya_eng,
    #     dlnz = -1, spec_type = 'N'
    # )
    # elec_exc_HeI_tf = tf.TransFuncAtRedshift(
    #     np.squeeze(rate_matrix_exc_HeI[:, np.where(eleceng > phys.He_exc_eng)]), 
    #     in_eng = eleceng, rs = rs*np.ones_like(eleceng), 
    #     eng = eleceng[eleceng > phys.He_exc_eng] - phys.He_exc_eng,
    #     dlnz = -1, spec_type = 'N'
    # )
    # elec_exc_HeII_tf = tf.TransFuncAtRedshift(
    #     np.squeeze(rate_matrix_exc_HeII[:, np.where(eleceng > 4*phys.lya_eng)]), 
    #     in_eng = eleceng, rs = rs*np.ones_like(eleceng), 
    #     eng = eleceng[eleceng > 4*phys.lya_eng] - 4*phys.lya_eng,
    #     dlnz = -1, spec_type = 'N'
    # )
    
    # Rebin these transfer functions back to eleceng.
    # elec_exc_HI_tf.rebin(eleceng)
    # elec_exc_HeI_tf.rebin(eleceng)
    # elec_exc_HeII_tf.rebin(eleceng)

    rate_vec_exc_HI = (
        (1 - xH)*phys.nH*rs**3 * phys.coll_exc_xsec(eleceng, species='HI') * beta_ele * phys.c
    )
    
    rate_vec_exc_HeI = (
        (phys.nHe/phys.nH - xHe)*phys.nH*rs**3 * phys.coll_exc_xsec(eleceng, species='HeI') * beta_ele * phys.c
    )
    
    rate_vec_exc_HeII = (
        xHe*phys.nH*rs**3 * phys.coll_exc_xsec(eleceng, species='HeII') * beta_ele * phys.c
    )

    elec_exc_HI_tf = tf.TransFuncAtRedshift(
        rate_vec_exc_HI[:, np.newaxis]*coll_exc_sec_elec_specs[0],
        in_eng = eleceng, rs = rs*np.ones_like(eleceng),
        eng = eleceng, dlnz = -1, spec_type  = 'N'
    )

    elec_exc_HeI_tf = tf.TransFuncAtRedshift(
        rate_vec_exc_HeI[:, np.newaxis]*coll_exc_sec_elec_specs[1],
        in_eng = eleceng, rs = rs*np.ones_like(eleceng),
        eng = eleceng, dlnz = -1, spec_type  = 'N'
    )

    elec_exc_HeII_tf = tf.TransFuncAtRedshift(
        rate_vec_exc_HeII[:, np.newaxis]*coll_exc_sec_elec_specs[2],
        in_eng = eleceng, rs = rs*np.ones_like(eleceng),
        eng = eleceng, dlnz = -1, spec_type  = 'N'
    )
   
    # Deposited energy for excitation.
    deposited_exc_vec = np.zeros_like(eleceng)
    
    #####################
    # Ionization  
    #####################
    
    # Construct the rate vector first. Secondary electron spectrum is an electron at in_eng - excitation energy, 
    # with a per second rate given by n*sigma*c.
    rate_vec_ion_HI = (
        (1 - xH)*phys.nH*rs**3 
        * phys.coll_ion_xsec(eleceng, species='HI') * beta_ele * phys.c
    )
    
    rate_vec_ion_HeI = (
        (phys.nHe/phys.nH - xHe)*phys.nH*rs**3 
        * phys.coll_ion_xsec(eleceng, species='HeI') * beta_ele * phys.c
    )
    
    rate_vec_ion_HeII = (
        xHe*phys.nH*rs**3 * 
        phys.coll_ion_xsec(eleceng, species='HeII') * beta_ele * phys.c
    )
    
    # Construct the spectra. 
    # elec_spec_ion_HI = np.array(
    #     [rate*phys.coll_ion_sec_elec_spec(in_eng, eleceng, species='HI') 
    # for rate,in_eng in zip(rate_vec_ion_HI,eleceng)]
    # )
    # elec_spec_ion_HeI = np.array(
    #     [rate*phys.coll_ion_sec_elec_spec(in_eng, eleceng, species='HeI') 
    # for rate,in_eng in zip(rate_vec_ion_HeI,eleceng)]
    # )
    # elec_spec_ion_HeII = np.array(
    #     [rate*phys.coll_ion_sec_elec_spec(in_eng, eleceng, species='HeII') 
    # for rate,in_eng in zip(rate_vec_ion_HeII,eleceng)]
    # )

    # transpose to force multiplication along first axis.
    elec_spec_ion_HI   = (
        rate_vec_ion_HI[:,np.newaxis]   * coll_ion_sec_elec_specs[0]
    )
    elec_spec_ion_HeI  = (
        rate_vec_ion_HeI[:,np.newaxis]  * coll_ion_sec_elec_specs[1]
    )
    elec_spec_ion_HeII = (
        rate_vec_ion_HeII[:,np.newaxis] * coll_ion_sec_elec_specs[2]
    )
    
    # Construct the TransFuncAtRedshift objects.
    # Electrons scatter from in_eng to in_eng - excitation energy.
    # Remove all of the columns (eng) that have energies below the 
    # excitation energy, 
    elec_ion_HI_tf = tf.TransFuncAtRedshift(
        elec_spec_ion_HI, in_eng = eleceng, rs = rs*np.ones_like(eleceng), 
        eng = eleceng, dlnz = -1, spec_type = 'N'
    )
    elec_ion_HeI_tf = tf.TransFuncAtRedshift(
        elec_spec_ion_HeI, in_eng = eleceng, rs = rs*np.ones_like(eleceng), 
        eng = eleceng, dlnz = -1, spec_type = 'N'
    )
    elec_ion_HeII_tf = tf.TransFuncAtRedshift(
        elec_spec_ion_HeII, in_eng = eleceng, rs = rs*np.ones_like(eleceng), 
        eng = eleceng, dlnz = -1, spec_type = 'N'
    )
    
    # Deposited energy for ionization.
    deposited_ion_vec = np.zeros_like(eleceng)
    
    #############################################
    # Heating
    #############################################
    
    dE_heat_dt = phys.elec_heating_engloss_rate(eleceng, xe, rs)
    
    deposited_heat_vec = np.zeros_like(eleceng)

    # new_eleceng = eleceng - dE_heat_dt

    # if not np.all(new_eleceng[1:] > eleceng[:-1]):
    #     utils.compare_arr([new_eleceng, eleceng])
    #     raise ValueError('heating loss is too large: smaller time step required.')

    # # After the check above, we can define the spectra by
    # # manually assigning slightly less than 1 particle along
    # # diagonal, and a small amount in the bin below. 

    # # N_n-1 E_n-1 + N_n E_n = E_n - dE_dt
    # # N_n-1 + N_n = 1
    # # therefore, (1 - N_n) E_n-1 - (1 - N_n) E_n = - dE_dt
    # # i.e. N_n = 1 + dE_dt/(E_n-1 - E_n)

    elec_heat_spec_grid = np.identity(eleceng.size)
    elec_heat_spec_grid[0,0] -= dE_heat_dt[0]/eleceng[0]
    elec_heat_spec_grid[1:, 1:] += np.diag(
        dE_heat_dt[1:]/(eleceng[:-1] - eleceng[1:])
    )
    elec_heat_spec_grid[1:, :-1] -= np.diag(
        dE_heat_dt[1:]/(eleceng[:-1] - eleceng[1:])
    )

    
    
    #############################################
    # Initialization of secondary spectra 
    #############################################

    # Low and high energy boundaries
    loweng = 3000
    eleceng_high = eleceng[eleceng > loweng]
    eleceng_high_ind = np.arange(eleceng.size)[eleceng > loweng]
    eleceng_low = eleceng[eleceng <= loweng]
    eleceng_low_ind  = np.arange(eleceng.size)[eleceng <= loweng]


    if eleceng_low.size == 0:
        raise TypeError('Energy abscissa must contain a low energy bin below 3 keV.')

    # Empty containers for quantities.
    # Final secondary photon spectrum.
    sec_phot_tf = tf.TransFuncAtRedshift(
        np.zeros((N_eleceng, N_photeng)), in_eng = eleceng,
        rs = rs*np.ones_like(eleceng), eng = photeng,
        dlnz = -1, spec_type = 'N'
    )
    # Final secondary low energy electron spectrum.
    sec_lowengelec_tf = tf.TransFuncAtRedshift(
        np.zeros((N_eleceng, N_eleceng)), in_eng = eleceng,
        rs = rs*np.ones_like(eleceng), eng = eleceng,
        dlnz = -1, spec_type = 'N'
    )

    # Start building sec_phot_tf and sec_lowengelec_tf.
    # Low energy regime first.

    sec_lowengelec_tf._grid_vals[:eleceng_low.size, :eleceng_low.size] = (
        np.identity(eleceng_low.size)
    )

    # Continuum energy loss rate per electron, dU_CMB/dt.
    CMB_upscatter_eng_rate = phys.thomson_xsec*phys.c*phys.CMB_eng_density(T)
    
    sec_elec_spec_N_arr = (
        elec_ICS_tf.grid_vals
        + elec_exc_HI_tf.grid_vals + elec_exc_HeI_tf.grid_vals + elec_exc_HeII_tf.grid_vals
        + elec_ion_HI_tf.grid_vals + elec_ion_HeI_tf.grid_vals + elec_ion_HeII_tf.grid_vals
        + elec_heat_spec_grid
    )
    
    sec_phot_spec_N_arr = phot_ICS_tf.grid_vals
    
    sec_elec_totN_arr = np.sum(sec_elec_spec_N_arr, axis=1)
    sec_elec_toteng_arr = np.dot(sec_elec_spec_N_arr, eleceng)
    sec_phot_toteng_arr = np.dot(sec_phot_spec_N_arr, photeng)
    
    deposited_ICS_eng_arr = (
        np.sum(elec_ICS_tf.grid_vals, axis=1)*eleceng
        - np.dot(elec_ICS_tf.grid_vals, eleceng)
        - (np.dot(sec_phot_spec_N_arr, photeng) - CMB_upscatter_eng_rate)
    )
    
    deposited_ICS_eng_arr[eleceng > 20*phys.me - phys.me] -= CMB_upscatter_eng_rate
    continuum_engloss_arr = CMB_upscatter_eng_rate*np.ones_like(eleceng)
    continuum_engloss_arr[eleceng > 20*phys.me - phys.me] = 0
    
    
    deposited_exc_eng_arr = (
        phys.lya_eng*np.sum(elec_exc_HI_tf.grid_vals, axis=1)
        + phys.He_exc_eng*np.sum(elec_exc_HeI_tf.grid_vals, axis=1)
        + 4*phys.lya_eng*np.sum(elec_exc_HeII_tf.grid_vals, axis=1)
    )
    
    deposited_ion_eng_arr = (
        phys.rydberg*np.sum(elec_ion_HI_tf.grid_vals, axis=1)/2
        + phys.He_ion_eng*np.sum(elec_ion_HeI_tf.grid_vals, axis=1)/2
        + 4*phys.rydberg*np.sum(elec_ion_HeII_tf.grid_vals, axis=1)/2
    )
    deposited_heat_eng_arr = dE_heat_dt
    
    # remove self-scattering, re-normalize. 
    np.fill_diagonal(sec_elec_spec_N_arr, 0)
    
    toteng_no_self_scatter_arr = (
        np.dot(sec_elec_spec_N_arr, eleceng)
        + np.dot(sec_phot_spec_N_arr, photeng)
        - continuum_engloss_arr
        + deposited_ICS_eng_arr
        + deposited_exc_eng_arr
        + deposited_ion_eng_arr
        + deposited_heat_eng_arr
    )
    
    fac_arr = eleceng/toteng_no_self_scatter_arr
    
    sec_elec_spec_N_arr *= fac_arr[:, np.newaxis]
    sec_phot_spec_N_arr *= fac_arr[:, np.newaxis]
    continuum_engloss_arr  *= fac_arr
    deposited_ICS_eng_arr  *= fac_arr
    deposited_exc_eng_arr  *= fac_arr
    deposited_ion_eng_arr  *= fac_arr
    deposited_heat_eng_arr *= fac_arr
    
    deposited_ICS_eng_arr[eleceng < loweng]  = 0
    deposited_exc_eng_arr[eleceng < loweng]  = 0
    deposited_ion_eng_arr[eleceng < loweng]  = 0
    deposited_heat_eng_arr[eleceng < loweng] = 0
    
    continuum_engloss_arr[eleceng < loweng]  = 0
    
    sec_phot_spec_N_arr[eleceng < loweng] = 0
    
    sec_lowengelec_N_arr = np.identity(eleceng.size)
    sec_lowengelec_N_arr[eleceng >= loweng] = 0
    sec_lowengelec_N_arr[eleceng_high_ind[0]:, :eleceng_high_ind[0]] += sec_elec_spec_N_arr[eleceng_high_ind[0]:, :eleceng_high_ind[0]]

    sec_highengelec_N_arr = np.zeros_like(sec_elec_spec_N_arr)
    sec_highengelec_N_arr[:, eleceng_high_ind[0]:] = sec_elec_spec_N_arr[:, eleceng_high_ind[0]:]
    
    print(np.allclose(sec_elec_spec_N_arr, np.tril(sec_elec_spec_N_arr)))
    
    # T = E.T + Prompt
    deposited_ICS_vec_linalg  = solve_triangular(np.identity(eleceng.size) - sec_elec_spec_N_arr, deposited_ICS_eng_arr, lower=True)
    deposited_exc_vec_linalg  = solve_triangular(np.identity(eleceng.size) - sec_elec_spec_N_arr, deposited_exc_eng_arr, lower=True)
    deposited_ion_vec_linalg  = solve_triangular(np.identity(eleceng.size) - sec_elec_spec_N_arr, deposited_ion_eng_arr, lower=True)
    deposited_heat_vec_linalg = solve_triangular(np.identity(eleceng.size) - sec_elec_spec_N_arr, deposited_heat_eng_arr, lower=True)
    
    continuum_engloss_vec_linalg = solve_triangular(np.identity(eleceng.size) - sec_elec_spec_N_arr, continuum_engloss_arr, lower=True)
    
    sec_phot_tf_linalg = solve_triangular(np.identity(eleceng.size) - sec_elec_spec_N_arr, sec_phot_spec_N_arr, lower=True)
    
    # Prompt: low energy e produced in secondary spectrum upon scattering (sec_lowengelec_N_arr).
    # T : high energy e produced (sec_highengelec_N_arr). 
    sec_lowengelec_tf_linalg = solve_triangular(np.identity(eleceng.size) - sec_highengelec_N_arr, sec_lowengelec_N_arr, lower=True)
    # High energy electron loop to get fully resolved spectrum.
    for i, eng in zip(eleceng_high_ind, eleceng_high):
        
        phot_ICS_N = phot_ICS_tf.grid_vals[i]
        
        elec_ICS_N      = elec_ICS_tf.grid_vals[i]
        
        elec_exc_HI_N   = elec_exc_HI_tf.grid_vals[i]
        elec_exc_HeI_N  = elec_exc_HeI_tf.grid_vals[i]
        elec_exc_HeII_N = elec_exc_HeII_tf.grid_vals[i]
        
        elec_ion_HI_N   = elec_ion_HI_tf.grid_vals[i]
        elec_ion_HeI_N  = elec_ion_HeI_tf.grid_vals[i]
        elec_ion_HeII_N = elec_ion_HeII_tf.grid_vals[i]
                
        # elec_heat_spec = spectools.rebin_N_arr(np.array([1]), np.array([eng]), eleceng)
        # elec_heat_spec.eng -= dE_heat_dt[i]
        # elec_heat_spec.rebin(eleceng)
        # elec_heat_N = elec_heat_spec.N

        elec_heat_N = elec_heat_spec_grid[i]        
        
        sec_elec_spec_N = sec_elec_spec_N_arr[i]
        sec_phot_spec_N = phot_ICS_N
        

        sec_elec_totN = sec_elec_totN_arr[i]
        # The *net* total energy of secondary electrons produced
        # per unit time.
        sec_elec_toteng = sec_elec_toteng_arr[i]
        # The total energy of secondary photons produced per unit time.
        sec_phot_toteng = sec_phot_toteng_arr[i]
        # Deposited ICS energy per unit time, dD/dt.
        # Numerical error (should be zero except for numerics)
        deposited_ICS_eng = deposited_ICS_eng_arr[i]
        # Deposited excitation energy. 
        deposited_exc_eng = deposited_exc_eng_arr[i]
        # Deposited ionization energy. Remember that the secondary spectrum
        # has two electrons for each ionization event.
        deposited_ion_eng = deposited_ion_eng_arr[i]
        # Deposited heating energy. 
        deposited_heat_eng = deposited_heat_eng_arr[i]
        
        # In the original code, the energy of the electron has gamma > 20,
        # then the continuum energy loss is assigned 
        # to deposited_eng instead.
        # I'm not sure if this is necessary, but let's be consistent with the
        # original code for now.

#         continuum_engloss = CMB_upscatter_eng_rate
        
#         if eng + phys.me > 20*phys.me:
#             deposited_ICS_eng -= CMB_upscatter_eng_rate
#             continuum_engloss = 0

        continuum_engloss = continuum_engloss_arr[i]

        # Remove self-scattering.
#         sec_elec_spec_N[i] = 0
        
        # Rescale.
#         toteng_no_self_scatter = (
#             np.dot(sec_elec_spec_N, eleceng)
#             + np.dot(sec_phot_spec_N, photeng)
#             - continuum_engloss
#             + deposited_ICS_eng
#             + deposited_exc_eng
#             + deposited_ion_eng
#             + deposited_heat_eng
#         )
        toteng_no_self_scatter = toteng_no_self_scatter_arr[i]
        
#         fac = eng/toteng_no_self_scatter
        # Normalize to one electron. 
        
#         sec_elec_spec_N    *= fac
#         sec_phot_spec_N    *= fac
#         continuum_engloss  *= fac
#         deposited_ICS_eng  *= fac
#         deposited_exc_eng  *= fac
#         deposited_ion_eng  *= fac
#         deposited_heat_eng *= fac

        # Get the full secondary photon spectrum. Type 'N'
        resolved_phot_spec_vals = np.dot(
            sec_elec_spec_N, sec_phot_tf.grid_vals
        )
        
        # Get the full secondary low energy electron spectrum. Type 'N'.

        resolved_lowengelec_spec_vals = np.dot(
            sec_elec_spec_N, sec_lowengelec_tf.grid_vals
        )
        
        # Add the resolved spectrum to the first scatter.
        sec_phot_spec_N += resolved_phot_spec_vals

        # Resolve the secondary electron continuum loss and deposition.
        continuum_engloss += np.dot(sec_elec_spec_N, cont_loss_ICS_vec)

        deposited_ICS_eng  += np.dot(sec_elec_spec_N, deposited_ICS_vec)
        deposited_exc_eng  += np.dot(sec_elec_spec_N, deposited_exc_vec)
        deposited_ion_eng  += np.dot(sec_elec_spec_N, deposited_ion_vec)
        deposited_heat_eng += np.dot(sec_elec_spec_N, deposited_heat_vec)

        # Now, append the resulting spectrum to the transfer function.
        # Do this without calling append of course: just add to the zeros
        # that fill the current row in _grid_vals.
        sec_phot_tf._grid_vals[i] += sec_phot_spec_N
        sec_lowengelec_tf._grid_vals[i] += resolved_lowengelec_spec_vals
        # Set the correct values in cont_loss_vec and deposited_vec.
        
        cont_loss_ICS_vec[i]  = continuum_engloss
        deposited_ICS_vec[i]  = deposited_ICS_eng
        deposited_exc_vec[i]  = deposited_exc_eng
        deposited_ion_vec[i]  = deposited_ion_eng
        deposited_heat_vec[i] = deposited_heat_eng

        
        failed_conservation_check = False
        
        if check_conservation_eng:

            conservation_check = (
                eng
                - np.dot(sec_lowengelec_tf.grid_vals[i], eleceng)
                + cont_loss_ICS_vec[i]
                - np.dot(sec_phot_tf.grid_vals[i], photeng)
                - deposited_exc_vec[i]
                - deposited_ion_vec[i]
                - deposited_heat_vec[i]
            )

            if np.abs(conservation_check/eng) > 0.1:
                failed_conservation_check = True
                print('failed!')
                
            if verbose or failed_conservation_check:
                
                print('***************************************************')
                print('rs: ', rs)
                print('injected energy: ', eng)
                print(
                    'Energy in low energy electrons: ',
                    np.dot(sec_lowengelec_tf.grid_vals[i], eleceng)
                )
                print(
                    'Energy in low energy electrons (linalg): ',
                    np.dot(sec_lowengelec_tf_linalg[i], eleceng)
                )
                utils.compare_arr([eleceng, sec_lowengelec_tf.grid_vals[i], sec_lowengelec_tf_linalg[i]])
                print('Energy in photons: ', np.dot(sec_phot_tf.grid_vals[i], photeng))
                print('Energy in photons (linalg): ', np.dot(sec_phot_tf_linalg[i], photeng))
                print('Continuum_engloss: ', cont_loss_ICS_vec[i])
                print('Continuum_engloss (linalg): ', continuum_engloss_vec_linalg[i])
                
                print(
                    'Energy in photons - Continuum: ',
                    np.dot(sec_phot_tf.grid_vals[i], photeng) - cont_loss_ICS_vec[i]
                )
                print(
                    'Deposited in ionization: ', deposited_ion_vec[i]
                )
                print(
                    'Deposited in ionization (linalg): ', deposited_ion_vec_linalg[i]
                )
                print(
                    'Deposited in excitation: ', deposited_exc_vec[i]
                )
                print(
                    'Deposited in excitation (linalg): ', deposited_exc_vec_linalg[i]
                )
                print(
                    'Deposited in heating: ', deposited_heat_vec[i]
                )
                print(
                    'Deposited in heating (linalg): ', deposited_heat_vec_linalg[i]
                )
                print(
                    'Energy is conserved up to (%): ',
                    conservation_check/eng*100
                )
                print('Deposited in ICS: ', deposited_ICS_vec[i])
                print('Deposited in ICS (linalg): ', deposited_ICS_vec_linalg[i])
                print(
                    'Energy conservation with deposited (%): ',
                    (conservation_check - deposited_ICS_vec[i])/eng*100
                )
                print('***************************************************')
                
            if failed_conservation_check:
                raise RuntimeError('Conservation of energy failed.')


    return (
        sec_phot_tf, sec_lowengelec_tf,
        deposited_ion_vec, deposited_exc_vec, deposited_heat_vec,
        cont_loss_ICS_vec, deposited_ICS_vec
    )

In [37]:
%lprun -f get_elec_cooling_tf_fast_linalg get_elec_cooling_tf_fast_linalg(ics_thomson_ref_tf, ics_rel_ref_tf, engloss_ref_tf, coll_ion_sec_elec_specs, coll_exc_sec_elec_specs, eleceng, photeng, rs, xH, xHe=xH*phys.nHe/phys.nH, ics_engloss_data = ics_engloss_data, check_conservation_eng=True, verbose=True)



True
***************************************************
rs:  3000
injected energy:  3016.701062885404
Energy in low energy electrons:  2845.118074807503
Energy in low energy electrons (linalg):  2845.118074807503
[[1.02978258e+00 1.10117805e-05 1.10117805e-05]
 [1.09203515e+00 1.16763852e-05 1.16763852e-05]
 [1.15805103e+00 1.23809513e-05 1.23809513e-05]
 ...
 [4.80630997e+12 0.00000000e+00 0.00000000e+00]
 [5.09686177e+12 0.00000000e+00 0.00000000e+00]
 [5.40497804e+12 0.00000000e+00 0.00000000e+00]]
Energy in photons:  2882.3712129754294
Energy in photons (linalg):  2882.3712129754294
Continuum_engloss:  2837.5054623630117
Continuum_engloss (linalg):  2837.5054623630117
Energy in photons - Continuum:  44.86575061241774
Deposited in ionization:  0.4707976377945742
Deposited in ionization (linalg):  0.4707976377945742
Deposited in excitation:  0.38909131851362727
Deposited in excitation (linalg):  0.38909131851362727
Deposited in heating:  125.92549553065011
Deposited in heating (lina

***************************************************
rs:  3000
injected energy:  13086.070328326896
Energy in low energy electrons:  2870.6769057544966
Energy in low energy electrons (linalg):  2870.6769057544966
[[1.02978258e+00 4.07400090e-04 4.07400090e-04]
 [1.09203515e+00 4.31988305e-04 4.31988305e-04]
 [1.15805103e+00 4.58054959e-04 4.58054959e-04]
 ...
 [4.80630997e+12 0.00000000e+00 0.00000000e+00]
 [5.09686177e+12 0.00000000e+00 0.00000000e+00]
 [5.40497804e+12 0.00000000e+00 0.00000000e+00]]
Energy in photons:  148184.6179120202
Energy in photons (linalg):  148184.6179120202
Continuum_engloss:  142354.10676372453
Continuum_engloss (linalg):  142354.10676372447
Energy in photons - Continuum:  5830.511148295685
Deposited in ionization:  17.51004337816073
Deposited in ionization (linalg):  17.510043378160724
Deposited in excitation:  15.167863630469634
Deposited in excitation (linalg):  15.167863630469633
Deposited in heating:  4355.35357123288
Deposited in heating (linalg):  435

Continuum_engloss (linalg):  278509.6047606334
Energy in photons - Continuum:  21724.946923671814
Deposited in ionization:  28.29641967792813
Deposited in ionization (linalg):  28.296419677928117
Deposited in excitation:  25.141565824398143
Deposited in excitation (linalg):  25.14156582439815
Deposited in heating:  6901.901452487984
Deposited in heating (linalg):  6901.901452487987
Energy is conserved up to (%):  -0.01939852374226448
Deposited in ICS:  -6.122715994542997
Deposited in ICS (linalg):  -6.122715994542998
Energy conservation with deposited (%):  6.314063224397457e-13
***************************************************
***************************************************
rs:  3000
injected energy:  33470.83240065101
Energy in low energy electrons:  2889.6929094369702
Energy in low energy electrons (linalg):  2889.69290943697
[[1.02978258e+00 6.72156867e-04 6.72156867e-04]
 [1.09203515e+00 7.12724207e-04 7.12724207e-04]
 [1.15805103e+00 7.55730776e-04 7.55730776e-04]
 ...
 [4.

Deposited in ICS:  -9.368289688451181
Deposited in ICS (linalg):  -9.36828968845118
Energy conservation with deposited (%):  3.99118185476575e-13
***************************************************
***************************************************
rs:  3000
injected energy:  80729.58080136002
Energy in low energy electrons:  2902.890196482205
Energy in low energy electrons (linalg):  2902.8901964822044
[[1.02978258e+00 8.52021889e-04 8.52021889e-04]
 [1.09203515e+00 9.03444798e-04 9.03444798e-04]
 [1.15805103e+00 9.57959659e-04 9.57959659e-04]
 ...
 [4.80630997e+12 0.00000000e+00 0.00000000e+00]
 [5.09686177e+12 0.00000000e+00 0.00000000e+00]
 [5.40497804e+12 0.00000000e+00 0.00000000e+00]]
Energy in photons:  505647.1389917446
Energy in photons (linalg):  505647.1389917446
Continuum_engloss:  436823.1954034825
Continuum_engloss (linalg):  436823.1954034822
Energy in photons - Continuum:  68823.94358826213
Deposited in ionization:  36.68653242433307
Deposited in ionization (linalg): 

***************************************************
rs:  3000
injected energy:  183614.85017674364
Energy in low energy electrons:  2910.6518941783847
Energy in low energy electrons (linalg):  2910.6518941783847
[[1.02978258e+00 9.57013381e-04 9.57013381e-04]
 [1.09203515e+00 1.01477294e-03 1.01477294e-03]
 [1.15805103e+00 1.07600547e-03 1.07600547e-03]
 ...
 [4.80630997e+12 0.00000000e+00 0.00000000e+00]
 [5.09686177e+12 0.00000000e+00 0.00000000e+00]
 [5.40497804e+12 0.00000000e+00 0.00000000e+00]]
Energy in photons:  741341.867236375
Energy in photons (linalg):  741341.8672363749
Continuum_engloss:  570903.5610125773
Continuum_engloss (linalg):  570903.5610125767
Energy in photons - Continuum:  170438.30622379773
Deposited in ionization:  41.21776880152776
Deposited in ionization (linalg):  41.21776880152771
Deposited in excitation:  37.83370361179806
Deposited in excitation (linalg):  37.83370361179804
Deposited in heating:  10199.349132779933
Deposited in heating (linalg):  10199.

...

Deposited in heating (linalg):  12544.642903395516
Energy is conserved up to (%):  -1.3630260217131076e-07
Deposited in ICS:  -19.62000375662149
Deposited in ICS (linalg):  -19.62000375662145
Energy conservation with deposited (%):  -6.7159866620967034e-15
***************************************************
***************************************************
rs:  3000
injected energy:  15264621479.240288
Energy in low energy electrons:  2917.531883088798
Energy in low energy electrons (linalg):  2917.531883088798
[[1.02978258e+00 1.04976467e-03 1.04976467e-03]
 [1.09203515e+00 1.11312214e-03 1.11312214e-03]
 [1.15805103e+00 1.18028916e-03 1.18028916e-03]
 ...
 [4.80630997e+12 0.00000000e+00 0.00000000e+00]
 [5.09686177e+12 0.00000000e+00 0.00000000e+00]
 [5.40497804e+12 0.00000000e+00 0.00000000e+00]]
Energy in photons:  15265503099.295727
Energy in photons (linalg):  15265503099.295727
Continuum_engloss:  897149.9019736792
Continuum_engloss (linalg):  897149.9019736787
Energy in photo

Energy in low energy electrons:  2917.531883104027
Energy in low energy electrons (linalg):  2917.531883104027
[[1.02978258e+00 1.04976467e-03 1.04976467e-03]
 [1.09203515e+00 1.11312214e-03 1.11312214e-03]
 [1.15805103e+00 1.18028916e-03 1.18028916e-03]
 ...
 [4.80630997e+12 0.00000000e+00 0.00000000e+00]
 [5.09686177e+12 0.00000000e+00 0.00000000e+00]
 [5.40497804e+12 0.00000000e+00 0.00000000e+00]]
Energy in photons:  62442181316.49031
Energy in photons (linalg):  62442181316.49031
Continuum_engloss:  897149.9019737426
Continuum_engloss (linalg):  897149.9019737421
Energy in photons - Continuum:  62441284166.58833
Deposited in ionization:  45.221184145490156
Deposited in ionization (linalg):  45.22118414549002
Deposited in excitation:  42.060806883767
Deposited in excitation (linalg):  42.06080688376691
Deposited in heating:  12544.817778350078
Deposited in heating (linalg):  12544.817778350081
Energy is conserved up to (%):  -3.142182075617944e-08
Deposited in ICS:  -19.62021061433

***************************************************
rs:  3000
injected energy:  214182800152.63754
Energy in low energy electrons:  2917.5318831055206
Energy in low energy electrons (linalg):  2917.5318831055206
[[1.02978258e+00 1.04976467e-03 1.04976467e-03]
 [1.09203515e+00 1.11312214e-03 1.11312214e-03]
 [1.15805103e+00 1.18028916e-03 1.18028916e-03]
 ...
 [4.80630997e+12 0.00000000e+00 0.00000000e+00]
 [5.09686177e+12 0.00000000e+00 0.00000000e+00]
 [5.40497804e+12 0.00000000e+00 0.00000000e+00]]
Energy in photons:  214183681772.42987
Energy in photons (linalg):  214183681772.4299
Continuum_engloss:  897149.901973749
Continuum_engloss (linalg):  897149.9019737486
Energy in photons - Continuum:  214182784622.5279
Deposited in ionization:  45.22118414635728
Deposited in ionization (linalg):  45.221184146357146
Deposited in excitation:  42.06080688489541
Deposited in excitation (linalg):  42.06080688489533
Deposited in heating:  12544.916053006207
Deposited in heating (linalg):  12544

***************************************************
rs:  3000
injected energy:  692797275386.8827
Energy in low energy electrons:  2917.53188310568
Energy in low energy electrons (linalg):  2917.53188310568
[[1.02978258e+00 1.04976467e-03 1.04976467e-03]
 [1.09203515e+00 1.11312214e-03 1.11312214e-03]
 [1.15805103e+00 1.18028916e-03 1.18028916e-03]
 ...
 [4.80630997e+12 0.00000000e+00 0.00000000e+00]
 [5.09686177e+12 0.00000000e+00 0.00000000e+00]
 [5.40497804e+12 0.00000000e+00 0.00000000e+00]]
Energy in photons:  692798234426.3926
Energy in photons (linalg):  692798234426.3926
Continuum_engloss:  897149.9019737495
Continuum_engloss (linalg):  897149.9019737489
Energy in photons - Continuum:  692797337276.4906
Deposited in ionization:  45.22118414645058
Deposited in ionization (linalg):  45.22118414645044
Deposited in excitation:  42.060806885021364
Deposited in excitation (linalg):  42.06080688502128
Deposited in heating:  12545.018231764725
Deposited in heating (linalg):  12545.0182

***************************************************
rs:  3000
injected energy:  2376396088830.3022
Energy in low energy electrons:  2917.531883105603
Energy in low energy electrons (linalg):  2917.531883105603
[[1.02978258e+00 1.04976467e-03 1.04976467e-03]
 [1.09203515e+00 1.11312214e-03 1.11312214e-03]
 [1.15805103e+00 1.18028916e-03 1.18028916e-03]
 ...
 [4.80630997e+12 0.00000000e+00 0.00000000e+00]
 [5.09686177e+12 0.00000000e+00 0.00000000e+00]
 [5.40497804e+12 0.00000000e+00 0.00000000e+00]]
Energy in photons:  2376829412312.7603
Energy in photons (linalg):  2376829412312.7603
Continuum_engloss:  897149.9019737492
Continuum_engloss (linalg):  897149.9019737488
Energy in photons - Continuum:  2376828515162.8584
Deposited in ionization:  45.22118414640565
Deposited in ionization (linalg):  45.22118414640551
Deposited in excitation:  42.06080688497072
Deposited in excitation (linalg):  42.06080688497063
Deposited in heating:  12545.177430145575
Deposited in heating (linalg):  12545

In [14]:
%autoreload

from darkhistory.electrons.elec_cooling import get_elec_cooling_tf_fast_linalg

%lprun -f get_elec_cooling_tf_fast get_elec_cooling_tf_fast(ics_thomson_ref_tf, ics_rel_ref_tf, engloss_ref_tf, coll_ion_sec_elec_specs, coll_exc_sec_elec_specs, eleceng, photeng, rs, xe, xHe=0, linalg=False, ics_engloss_data = ics_engloss_data, check_conservation_eng=True, verbose=True)




***************************************************
rs:  2750
injected energy:  3016.701062885404
Energy in low energy electrons:  2846.294789387355
Energy in photons:  2806.7889781466815
Continuum_engloss:  2763.099252286273
Energy in photons - Continuum:  43.689725860408544
Deposited in ionization:  1.2852967581070838
Deposited in excitation:  0.26550088845996717
Deposited in heating:  125.22734744635872
Energy is conserved up to (%):  -0.0020418813134401807
Deposited in ICS:  -0.06159745528587722
Energy conservation with deposited (%):  1.5541258339431474e-14
***************************************************
***************************************************
rs:  2750
injected energy:  3199.0671427921043
Energy in low energy electrons:  2848.2290043079074
Energy in photons:  5793.4433458984895
Continuum_engloss:  5700.491910679315
Energy in photons - Continuum:  92.95143521917453
Deposited in ionization:  2.6268515664054632
Deposited in excitation:  0.5429899298795923
Deposited i

Deposited in ionization:  100.71400971410223
Deposited in excitation:  21.402488136039317
Deposited in heating:  8567.374852254072
Energy is conserved up to (%):  -0.013463839725541438
Deposited in ICS:  -8.594830467734848
Energy conservation with deposited (%):  -3.278264470433448e-13
***************************************************
***************************************************
rs:  2750
injected energy:  67695.44703762903
Energy in low energy electrons:  3041.9199054192377
Energy in photons:  459212.2452972698
Continuum_engloss:  403358.1498221618
Energy in photons - Continuum:  55854.095475107955
Deposited in ionization:  102.11764149148846
Deposited in excitation:  21.707589617520206
Deposited in heating:  8684.418635195821
Energy is conserved up to (%):  -0.013017432617129602
Deposited in ICS:  -8.812209202855506
Energy conservation with deposited (%):  -1.9574302348316703e-13
***************************************************
********************************************

***************************************************
***************************************************
rs:  2750
injected energy:  629825.8196346217
Energy in low energy electrons:  3099.1628196344927
Energy in photons:  1351120.7475986562
Continuum_engloss:  736089.5992036804
Energy in photons - Continuum:  615031.1483949758
Deposited in ionization:  129.88527294195234
Deposited in excitation:  27.79509747221666
Deposited in heating:  11553.887703941038
Energy is conserved up to (%):  -0.002549856459244977
Deposited in ICS:  -16.059654343059492
Energy conservation with deposited (%):  -1.4075795766319343e-13
***************************************************
***************************************************
rs:  2750
injected energy:  667900.1476361318
Energy in low energy electrons:  3099.635812006375
Energy in photons:  1395750.494565425
Continuum_engloss:  742689.4983066565
Energy in photons - Continuum:  653060.9962587685
Deposited in ionization:  130.1140569032339
Deposited i

***************************************************
rs:  2750
injected energy:  2290995.2953919168
Energy in low energy electrons:  3104.397342403809
Energy in photons:  3120633.1835225895
Continuum_engloss:  845141.4911522736
Energy in photons - Continuum:  2275491.692370316
Deposited in ionization:  132.41667663802676
Deposited in excitation:  28.358590502713128
Deposited in heating:  12256.877151825853
Energy is conserved up to (%):  -0.0008051845329771508
Deposited in ICS:  -18.44673976912303
Energy conservation with deposited (%):  -2.648940393372595e-14
***************************************************
***************************************************
rs:  2750
injected energy:  2429490.8978384337
Energy in low energy electrons:  3104.475072801442
Energy in photons:  3262342.4080005474
Continuum_engloss:  848375.6804660506
Energy in photons - Continuum:  2413966.7275344967
Deposited in ionization:  132.4542575782152
Deposited in excitation:  28.367001297671468
Deposited in he

***************************************************
rs:  2750
injected energy:  21314993.147134572
Energy in low energy electrons:  3105.20031066054
Energy in photons:  22192496.256532654
Continuum_engloss:  893383.3991054808
Energy in photons - Continuum:  21299112.857427172
Deposited in ionization:  132.8048744095113
Deposited in excitation:  28.445704532174204
Deposited in heating:  12633.337412107825
Energy is conserved up to (%):  -9.147830439071345e-05
Deposited in ICS:  -19.49859431116004
Energy conservation with deposited (%):  -3.919470330850012e-15
***************************************************
***************************************************
rs:  2750
injected energy:  22603530.414318632
Energy in low energy electrons:  3105.201772956274
Energy in photons:  23481030.36120466
Continuum_engloss:  893383.4041698467
Energy in photons - Continuum:  22587646.957034815
Deposited in ionization:  132.8055813050788
Deposited in excitation:  28.44586447475625
Deposited in heati

...

***************************************************
rs:  2750
injected energy:  680234814.24638
Energy in low energy electrons:  3105.2141356382695
Energy in photons:  681112260.8386443
Continuum_engloss:  893383.4470037281
Energy in photons - Continuum:  680218877.3916405
Deposited in ionization:  132.8115575821998
Deposited in excitation:  28.447219538489854
Deposited in heating:  12689.880643075741
Energy is conserved up to (%):  -2.8664831624776945e-06
Deposited in ICS:  -19.498816365326174
Energy conservation with deposited (%):  -7.402992227888407e-15
***************************************************
***************************************************
rs:  2750
injected energy:  721356474.5979489
Energy in low energy electrons:  3105.2141375679275
Energy in photons:  722233921.0711935
Continuum_engloss:  893383.4470104175
Energy in photons - Continuum:  721340537.624183
Deposited in ionization:  132.811558515016
Deposited in excitation:  28.447219751993604
Deposited in heating:

Energy is conserved up to (%):  -3.8963232085815563e-07
Deposited in ICS:  -19.49891647380484
Energy conservation with deposited (%):  -3.222127587550206e-14
***************************************************
***************************************************
rs:  2750
injected energy:  5306969826.598106
Energy in low energy electrons:  3105.2141536583813
Energy in photons:  5307847271.248327
Continuum_engloss:  893383.4470662038
Energy in photons - Continuum:  5306953887.801261
Deposited in ionization:  132.81156629328822
Deposited in excitation:  28.447221534638373
Deposited in heating:  12691.822819088795
Energy is conserved up to (%):  -3.674208711851539e-07
Deposited in ICS:  -19.498912315313888
Energy conservation with deposited (%):  -4.626191984324368e-14
***************************************************
***************************************************
rs:  2750
injected energy:  5627787588.545746
Energy in low energy electrons:  3105.2141537003363
Energy in photons:  56

Continuum_engloss:  893383.4470676326
Energy in photons - Continuum:  36817308421.335396
Deposited in ionization:  132.81156649234663
Deposited in excitation:  28.447221580474952
Deposited in heating:  12692.184960675919
Energy is conserved up to (%):  -5.29618555713388e-08
Deposited in ICS:  -19.49912680272857
Energy conservation with deposited (%):  -3.082871415742206e-14
***************************************************
***************************************************
rs:  2750
injected energy:  39043010955.325554
Energy in low energy electrons:  3105.214154071684
Energy in photons:  39043888399.607956
Continuum_engloss:  893383.4470676377
Energy in photons - Continuum:  39042995016.16089
Deposited in ionization:  132.81156649308116
Deposited in excitation:  28.44722158064463
Deposited in heating:  12692.190843412685
Energy is conserved up to (%):  -4.9942673199686486e-08
Deposited in ICS:  -19.49912276753702
Energy conservation with deposited (%):  -1.539837735854298e-15
*****

Deposited in excitation:  28.4472215823387
Deposited in heating:  12692.383703299054
Energy is conserved up to (%):  -4.244760449355141e-09
Deposited in ICS:  -19.499370576344184
Energy conservation with deposited (%):  -1.1841968592885253e-14
***************************************************
***************************************************
rs:  2750
injected energy:  487146624199.9802
Energy in low energy electrons:  3105.2141540867724
Energy in photons:  487147501644.0658
Continuum_engloss:  893383.4470676903
Energy in photons - Continuum:  487146608260.6187
Deposited in ionization:  132.81156650037505
Deposited in excitation:  28.447221582332315
Deposited in heating:  12692.388122077426
Energy is conserved up to (%):  -4.0028135933340795e-09
Deposited in ICS:  -19.49942005321975
Energy conservation with deposited (%):  -3.1046037810510156e-14
***************************************************
***************************************************
rs:  2750
injected energy:  51659

rs:  2750
injected energy:  4532321378990.888
Energy in low energy electrons:  3105.2141540871908
Energy in photons:  4533754033329.9375
Continuum_engloss:  893383.4470676916
Energy in photons - Continuum:  4533753139946.49
Deposited in ionization:  132.81156650057738
Deposited in excitation:  28.447221582380052
Deposited in heating:  12692.743319444464
Energy is conserved up to (%):  -0.03159036606396381
Deposited in ICS:  -1431776914.818393
Energy conservation with deposited (%):  -2.6670266624253703e-15
***************************************************
***************************************************
rs:  2750
injected energy:  4806309972999.415
Energy in low energy electrons:  3105.2141540870775
Energy in photons:  4807704537344.525
Continuum_engloss:  893383.4470676915
Energy in photons - Continuum:  4807703643961.078
Deposited in ionization:  132.81156650052176
Deposited in excitation:  28.447221582367074
Deposited in heating:  12692.72725839125
Energy is conserved up to (%)

In [164]:
%%prun
for i in np.arange(120):
    get_elec_cooling_tf_fast(
        ics_thomson_ref_tf, ics_rel_ref_tf, engloss_ref_tf, 
        coll_ion_sec_elec_specs, coll_exc_sec_elec_specs, 
        eleceng, photeng, rs, xe, xHe=0, 
        linalg=False, ics_engloss_data = ics_engloss_data, 
        check_conservation_eng=False, verbose=False
    )
    get_elec_cooling_tf_fast_linalg(
        ics_thomson_ref_tf, ics_rel_ref_tf, engloss_ref_tf, 
        coll_ion_sec_elec_specs, coll_exc_sec_elec_specs, 
        eleceng, photeng, rs, xe, xHe=0, 
        ics_engloss_data = ics_engloss_data, 
        check_conservation_eng=False, verbose=False
    )